In [ ]:
import os
import pandas as pd
import numpy as np
import random


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')

from sklearn.cluster import KMeans
from sklearn.model_selection import cross_validate, train_test_split, ShuffleSplit, learning_curve, validation_curve
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix,\
                            silhouette_score, v_measure_score as vscr
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, normalize, MinMaxScaler, LabelEncoder
from sklearn.mixture import GaussianMixture as EM
import itertools
import timeit


In [ ]:
# from imblearn.over_sampling import RandomOverSampler
from IPython.display import display
np.random.seed(1)

In [ ]:
# import dill
# # # # from dill.settings import settings
# dill.dump_session('avila-ntf.db')
# # dill_file = open("avila-ntf.db", "rb")
# dill.load(dill_file)

Data2

In [ ]:
df_avila = pd.read_csv('avila-pro4.csv')
print("Data has",len(df_avila),"rows and", len(df_avila.columns),"columns.")
if df_avila.isnull().values.any():
    print("missing Data")
# df_avila.head()
df_avila.describe(include='all')

In [ ]:
np.unique(df_avila.y)

In [ ]:
# sns.pairplot(df_wilt)


In [ ]:
np.bincount(df_avila.y)

In [ ]:
df_avila.y.value_counts()

In [ ]:
# plt.title("Counts of Wilt Data Labels")
# sns.countplot(df_wilt['class'])

In [ ]:
# min_max_scaler = preprocessing.MinMaxScaler()
# names.insert(0, names.pop(names.index('y')))
# df_num = df_clean[names[1:]]
# x_scaled = min_max_scaler.fit_transform(df_num)
# col_names = df_wilt.columns
# print(col_names)

In [ ]:
# scalar = MinMaxScaler()

In [ ]:
# df_scaled = pd.DataFrame(x_scaled, columns=names[1:])
# df_scaled.head()
# rem_names = ['genergy', 'gpuls', 'gdenergy',
#        'gdpuls', 'nbumps', 'nbumps2', 'nbumps3', 'nbumps4',
#        'nbumps5', 'nbumps6', 'nbumps7', 'nbumps89', 'energy', 'maxenergy',
#        'class']


# scalar.fit(df_wilt[col_names[1:]])

# x_scaled = scalar.transform(df_wilt[col_names[1:]])
# x_scaled_test = scalar.transform(df_wilt_test[col_names[1:]])

# df_sc = pd.DataFrame(x_scaled, columns=col_names[1:-1])

In [ ]:
def get_av_data():


    x = np.array(df_avila.values[:,1:])
    y = np.array(df_avila.values[:,0])

    return x, y



In [ ]:
avX, avY = get_av_data()
X_train, X_test, y_train, y_test = train_test_split(np.array(avX),np.array(avY), test_size=0.20)

In [ ]:
# df_clean.sort_index(axis=1, inplace=True)

In [ ]:
# df_clean.head()

In [ ]:
# sns.countplot(x='class',data=df_clean)

In [ ]:
# np.bincount(y_train), np.bincount(y_test)

In [ ]:
# sns.countplot(df_wilt['class'])

In [ ]:
# sns.countplot(y_test)

In [ ]:
# #Export
# df_wilt.to_csv('wilt-tr-pro.csv', index=None)
# df_wilt_test.to_csv('wilt-test-pro.csv', index=None)

### Resampling done only to training data. Testing data is still imbalanced

In [ ]:
# sns.countplot(y_test)


Now we have successfully loaded and processed both datasets. We are ready to start the ML!

### helper funcs

In [ ]:
#src: sklearn
def pllc(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 20)):

    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("F1")
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=-1, train_sizes=train_sizes, scoring='f1')
    


    
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    
    results ={'sizes':train_sizes,'tr_scores':train_scores_mean, 'val_scores': test_scores_mean, 'title':title}
    
    
    
    
    
    plt.grid(True)

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="g")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="r")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="g",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="r",
             label="Cross-validation score")

    plt.legend(loc="best")
    plt.tight_layout()
    
    return plt, results

In [ ]:

from sklearn.neural_network import MLPClassifier


def tune_hp(estimator, X_train, y_train, title, param_name,param_range, xlabel,xvals=None, cv=5):

    
            
            
    train_scores, val_scores = validation_curve(estimator, X_train, y_train, param_name, 
                                                 param_range, cv=cv, n_jobs=-1, scoring='f1')
    
    train_scores_mean = np.mean(train_scores, axis=1)
    #train_scores_std = np.std(train_scores, axis=1)
    val_scores_mean = np.mean(val_scores, axis=1)
    #test_scores_std = np.std(test_scores, axis=1)
    
    if xvals is not None:
        param_range=xvals
            
    plt.grid(True)  
    plt.plot(param_range, train_scores_mean, 'o-', color = 'g', label='Train Score')
    plt.plot(param_range, val_scores_mean, 'o-', color='r', label='Validation Score')
    plt.ylabel('F1')
    plt.xlabel(xlabel)
    
    plt.title(title)
    plt.legend(loc='best')
#     plt.tight_layout()

# Clustering

In [ ]:
def kmeans_expt(X,title, validate=False, y=None):

    kclusters = list(np.arange(1,10,1))
    sse, sil_scores, train_times, = [], [], []
    v_scores = []

    for k in kclusters:
        start_time = timeit.default_timer()
        km = KMeans(n_clusters=k, n_init=10,random_state=1,n_jobs=-1).fit(X)
        sse.append(km.inertia_)
        end_time = timeit.default_timer()
        train_times.append(end_time - start_time)
        if k != 1:
            sil_scores.append(silhouette_score(X, km.labels_))
            if validate:
                v_scores.append(vscr(y, km.labels_))
            
        

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(kclusters, sse)
    plt.grid(True)
    plt.xlabel('No. Clusters')
    plt.ylabel('SSE')
    plt.title('Elbow Plot for KMeans: '+ title)
    plt.show()
    
    
    

    fig = plt.figure()
    ax = fig.add_subplot(111)
    sns.barplot(x=kclusters[1:],y=sil_scores, color='royalblue')
    plt.grid(True)
    plt.xlabel('No. Clusters')
    plt.ylabel('Avg Silhouette Score')
    plt.title('Avg Silhouette Score for KMeans: '+ title)
    plt.show()
    
    if validate:
        print("Validation: Using Training Labels")
        
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.plot(kclusters[1:],v_scores)
        plt.grid(True)
        plt.xlabel('No. Clusters')
        plt.ylabel('V Score')
        plt.title('V Score for KMeans: '+ title)
        plt.show()
        
    

In [ ]:

kmeans_expt(X_train,'Avila Data', validate=True, y=y_train)

In [ ]:
def EM_expt(X,title, validate=False, y = None):

    klist = list(np.arange(1,10,1))
    sil_scores, train_times, aic_scores, bic_scores, v_scores = [], [], [], [], []

    for k in klist:
        start_time = timeit.default_timer()
        em = EM(n_components=k,covariance_type='diag',n_init=1,warm_start=True,random_state=1).fit(X)
#         sse.append(em.inertia_)
        end_time = timeit.default_timer()
        train_times.append(end_time - start_time)
        aic_scores.append(em.aic(X))
        bic_scores.append(em.bic(X))
        labels = em.predict(X)
        if k != 1:
            sil_scores.append(silhouette_score(X,labels))
            if validate:
                v_scores.append(vscr(y,labels))
    
    
    

    fig = plt.figure()
    ax = fig.add_subplot()
    sns.barplot(x=klist[1:],y=sil_scores, color='royalblue')
    plt.grid(True)
    plt.xlabel('No. Clusters')
    plt.ylabel('Avg Silhouette Score')
    plt.title('Avg Silhouette Score for EM: '+ title)
    plt.show()
    
    
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     sns.barplot(x=klist[1:],y=dbs_scores)
#     plt.grid(True)
#     plt.xlabel('No. Clusters')
#     plt.ylabel('DBS Score')
#     plt.title('DBS Score for EM: '+ title)
#     plt.show()
    
    
    
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     sns.barplot(x=klist[1:],y=chs_scores)
#     plt.grid(True)
#     plt.xlabel('No. Clusters')
#     plt.ylabel('CHS Score')
#     plt.title('CHS Score for EM: '+ title)
#     plt.show()
    

    fig = plt.figure()
    ax = fig.add_subplot()
    ax.plot(klist, aic_scores, label='AIC')
    ax.plot(klist, bic_scores,label='BIC')
    plt.grid(True)
    plt.xlabel('No. Distributions')
    plt.ylabel('Model Complexity Score')
    plt.title('EM Model Complexity: '+ title)
    plt.legend(loc="best")
    plt.show()
    
    
    if validate:
        print("Validation:  using Training labels")
        fig = plt.figure()
        ax = fig.add_subplot()
        ax.plot(klist[1:], v_scores)
        plt.grid(True)
        plt.xlabel('No. Distributions')
        plt.ylabel('V Score')
        plt.title('EM Model V scores: '+ title)
        plt.legend(loc="best")
        plt.show()

In [ ]:
EM_expt(X_train,'Avila Data', True, y_train )

In [ ]:
from sklearn.decomposition import PCA, FastICA as ICA, FactorAnalysis, NMF
from sklearn.random_projection import GaussianRandomProjection as GRP
from itertools import product
from collections import defaultdict

In [ ]:
def run_PCA(X,title):
    
    pca = PCA(random_state=1).fit(X) #for all components
    
#     xtrain_pca = pca.transform(X)
#     projected_pca = pca.inverse_transform(xtrain_pca)
#     loss = ((X_train - projected_pca) ** 2).mean()
#     print(loss)
   
    print(pca.explained_variance_)
    print(pca.explained_variance_ratio_)
#     print(pca.n_components_ )
    
    cum_var = np.cumsum(pca.explained_variance_ratio_)
    
    fig, ax1 = plt.subplots()
    ax1.plot(list(range(1,len(pca.explained_variance_ratio_)+1)), cum_var, 'r-')
    ax1.set_xlabel('Principal Components')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax1.set_ylabel('Cumulative Explained Variance Ratio', color='r')
    ax1.tick_params('y', colors='r')
    plt.grid(False)

    ax2 = ax1.twinx()
    ax2.plot(list(range(1,len(pca.singular_values_)+1)), pca.singular_values_, 'k-')
    ax2.set_ylabel('Eigenvalues')
    ax2.tick_params('y', colors='k')
    plt.grid(False)

    plt.title("PCA Expl. variance and eigenvalues: "+ title)
    fig.tight_layout()
    plt.show()


In [ ]:
run_PCA(X_train,'Avila Data')

In [ ]:
def rec_test(X, m=PCA, title="PCA"):
    rg = range(1,X.shape[1]+1)
    re_err = []
    pca = m(random_state=1)
    for i in rg:
        pca.set_params(n_components=i)
        pca.fit(X) #for all components
    #     print(pca.components_)

        xtrain_pca = pca.transform(X)
        projected_pca = pca.inverse_transform(xtrain_pca)
        loss = ((X_train - projected_pca) ** 2).mean()
        re_err.append(loss)
        print(loss)
#         print(loss1)
    
    
    plt.plot(list(rg), re_err, color='royalblue')
    plt.xlabel('No. Components')
    plt.ylabel('Reconstruction Error')
    plt.title(title + ' reconstruction error')

In [ ]:
rec_test(X_train, m=PCA, title='PCA Avila' )

In [ ]:

def run_ICA(X,title):
    feat_len = list(np.arange(2,(X.shape[1]-1),3))
    feat_len.append(X.shape[1])
    ica = ICA(random_state=1)
    kurt = []

    for comp in feat_len:
        ica.set_params(n_components=comp)
        tmp = ica.fit_transform(X)
        tmp = pd.DataFrame(tmp)
        tmp = tmp.kurt(axis=0)
        kurt.append(tmp.abs().mean())

    plt.figure()
    plt.title("ICA Kurtosis: "+ title)
    plt.xlabel("Independent Components")
    plt.ylabel("Avg Kurtosis Across IC")
    plt.plot(feat_len, kurt, 'b-')
    plt.grid(False)
    plt.show()


In [ ]:
run_ICA(X_train,'Avila Data')

In [ ]:
rec_test(X_train,m=ICA, title='Avila ICA')

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
def error_dist_cal(X1,X2):
    assert X1.shape[0] == X2.shape[0]
    
    d1 = pairwise_distances(X1)
    d2 = pairwise_distances(X2)
    loss = (np.mean((d1-d2)**2))
    
#     np.corrcoef(d1.ravel(),d2.ravel())[0,1]
    return loss

def run_GRP(X,title):
    
    feat_len = list(np.arange(2,(X.shape[1]),1))
#     feat_len.append(X.shape[1])
    tmp = defaultdict(dict)

    for i,comp in product(range(3),feat_len):
        rp = GRP(random_state=i, n_components=comp)
        tmp[comp][i] = error_dist_cal(rp.fit_transform(X), X)
    tmp = pd.DataFrame(tmp).T
    mean_recon = tmp.mean(axis=1).tolist()


    fig, ax1 = plt.subplots()
    ax1.plot(feat_len,mean_recon, 'b-')
    ax1.set_xlabel('Random Components')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax1.set_ylabel('Reconstruction Error', color='b')
    ax1.tick_params('y', colors='b')
    plt.grid(False)


    plt.title("Random Components with 3 Restarts: "+ title)
    fig.tight_layout()
    plt.show()
    

In [ ]:
run_GRP(X_train, 'Avila Data')

In [ ]:
def grp_testf(X):
    rg = range(2,X.shape[1]+1)
    re_err = []
    pca = GRP(random_state=1)
    for i in rg:
        pca.set_params(n_components=i)
        pca.fit(X) #for all components
    #     print(pca.components_)

        xtrain_pca = pca.transform(X)
#         projected_pca = pca.inverse_transform(xtrain_pca)
        projected_pca = np.dot(xtrain_pca, pca.components_)
    
    
#         loss = ((X_train - projected_pca) ** 2).mean()

        loss = ((X_train - projected_pca) ** 2).mean()
        re_err.append(loss)
        print(loss)
    
    
    sns.barplot(x=list(rg), y=re_err, color='royalblue')

In [ ]:
grp_testf(X_train)

In [ ]:
rec_test(X_train, m=NMF, title='NMF Avila')

In [ ]:
pca1 = PCA(n_components=5,random_state=1).fit(X_train)
ica1 = ICA(n_components=8,random_state=1).fit(X_train)
grp1 = GRP(n_components=6,random_state=1).fit(X_train)
nmf1 = NMF(n_components=5,random_state=1).fit(X_train)

In [ ]:
ica2 = ICA(n_components=5,random_state=1).fit(X_train)
ica_train2 = ica2.transform(X_train)
ica_test2 = ica2.transform(X_test)

In [ ]:
pca_train = pca1.transform(X_train)
ica_train = ica1.transform(X_train)
grp_train = grp1.transform(X_train)
nmf_train = nmf1.transform(X_train)

In [ ]:
#applying D-reductions to test data (for later use)
pca_test = pca1.transform(X_test)
ica_test = ica1.transform(X_test)
grp_test = grp1.transform(X_test)
nmf_test = nmf1.transform(X_test)

In [ ]:
kmeans_expt(pca_train,'PCA Avila Data', validate=True, y=y_train)
kmeans_expt(ica_train,'ICA Avila Data', validate=True, y=y_train)
kmeans_expt(grp_train,'GRP Avila Data', validate=True, y=y_train)
kmeans_expt(nmf_train,'NMF Avila Data', validate=True, y=y_train)

In [ ]:
EM_expt(pca_train,'PCA Avila Data', validate=True, y=y_train)
EM_expt(ica_train,'ICA Avila Data', validate=True, y=y_train)
EM_expt(grp_train,'GRP Avila Data', validate=True, y=y_train)
EM_expt(nmf_train,'NMF Avila Data', validate=True, y=y_train)

# Neural Network

In [ ]:
mlp =  MLPClassifier(solver='adam',random_state=1, verbose=0, hidden_layer_sizes=(100,200,80,),
                          learning_rate_init= 0.01, activation= 'relu')

In [ ]:
def testing_func(clf,X_train, X_test, y_train, y_test, title):
    print("Testing for: " + title)
    
    start_time = timeit.default_timer()
    clf.fit(X_train, y_train)
    end_time = timeit.default_timer()
    training_time = end_time - start_time
    
    start_time = timeit.default_timer()    
    y_pred = clf.predict(X_test)
    end_time = timeit.default_timer()
    pred_time = end_time - start_time
    

    f1 = f1_score(y_test,y_pred, average='macro')
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred, average='macro')
    recall = recall_score(y_test,y_pred, average='macro')
    cm = confusion_matrix(y_test,y_pred )
    
    df_cm = pd.DataFrame(cm, index= ['A', 'E', 'F', 'I'], columns=['A', 'E', 'F', 'I'])


    
    print("Model Training Time (s):   "+"{:.5f}".format(training_time))
    print("Model Prediction Time (s): "+"{:.5f}\n".format(pred_time))
    print("F1 Score:  "+"{:.2f}".format(f1))
    print("Accuracy:  "+"{:.2f}".format(accuracy))

    print("Precision: "+"{:.2f}".format(precision))
    print("Recall:  "+"{:.2f}".format(recall))


    ax = sns.heatmap(df_cm,cmap="YlGnBu", annot=True, fmt="d")
    plt.title(title)
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    plt.show()
    


In [ ]:
testing_func(mlp,X_train, X_test, y_train, y_test, "Avila original Data")

In [ ]:
testing_func(mlp,pca_train, pca_test, y_train, y_test, "Avila PCA Data")
testing_func(mlp,ica_train, ica_test, y_train, y_test, "Avila ICA Data")
testing_func(mlp,grp_train, grp_test, y_train, y_test, "Avila GRP Data")
testing_func(mlp,nmf_train, nmf_test, y_train, y_test, "Avila NMF Data")

# Add Cluster Labels

In [ ]:
def add_feat(X_tr, Xt,km_n,em_n):
    km = KMeans(n_clusters=km_n,random_state=1,n_jobs=-1).fit(X_tr)
    em = EM(n_components=em_n,covariance_type='diag',warm_start=True,random_state=1).fit(X_tr)
    if km_n > 2:
        km_tr = np.eye(km_n)[km.labels_].astype('int')
        km_t = np.eye(km_n)[km.predict(Xt)].astype('int')
    else:
         
        km_tr = km.labels_.reshape(-1,1).astype('int')
        km_t = km.predict(Xt).reshape(-1,1).astype('int')
        
        
        
    if em_n > 2:
        em_tr = np.eye(em_n)[em.predict(X_tr)].astype('int')
        em_t = np.eye(em_n)[em.predict(Xt)].astype('int')
    else:
         
        em_tr = em.predict(X_tr).reshape(-1,1).astype('int')
        em_t = em.predict(Xt).reshape(-1,1).astype('int')

    
    km_added_tr = np.hstack((X_tr, km_tr ))
    X_tr = np.hstack((km_added_tr, em_tr ))

    km_added_t = np.hstack((Xt, km_t ))
    X_t = np.hstack((km_added_t, em_t ))
    
    
    
    
    
    return X_tr, X_t

In [ ]:
X_train_p, X_test_p = add_feat(X_train, X_test,4,4)
pca_p, pca_tp = add_feat(pca_train,pca_test,4,4)
ica_p, ica_tp = add_feat(ica_train,ica_test,3,4)
grp_p, grp_tp = add_feat(grp_train,grp_test,3,3)
nmf_p, nmf_tp = add_feat(nmf_train,nmf_test,4,4)

In [ ]:
testing_func(mlp,X_train_p, X_test_p, y_train, y_test, "original Data + Clusters")

In [ ]:
testing_func(mlp,pca_p, pca_tp, y_train, y_test, "Avila PCA Data + Clusters")
testing_func(mlp,ica_p, ica_tp, y_train, y_test, "Avila ICA Data + Clusters")
testing_func(mlp,grp_p, grp_tp, y_train, y_test, "Avila GRP Data + Clusters")
testing_func(mlp,nmf_p, nmf_tp, y_train, y_test, "Avila NMF Data + Clusters")

In [ ]:
temp = pd.DataFrame(X_train)

In [ ]:
sns.pairplot(temp)

In [ ]:
plt.scatter(temp.iloc[:,4], temp.iloc[:,1] )

In [ ]:
# def kmeans_expt1(X,title, validate=False, y=None):

#     kclusters = list(np.arange(1,10,1))
#     sse, sil_scores, train_times, = [], [], []
#     v_scores = []

# #     for k in kclusters:
   
#     km = KMeans(n_clusters=4,random_state=1,n_jobs=-1).fit(X)

#     temp = pd.DataFrame(km.labels_)
#     print(temp.columns)
# #     temp
#     sns.lmplot(data=temp,x=1,y=1, hue='label', fit_reg=False, legend=True, legend_out=True)
#     plt.show()
    

    

In [ ]:
# kmeans_expt1(pca_train,'PCA Avila Data')

In [ ]:
# kmeans_expt(pca_train,'PCA Avila Data')
# kmeans_expt(ica_train,'ICA Avila Data')
# kmeans_expt(grp_train,'GRP Avila Data')
# kmeans_expt(nmf_train,'NMF Avila Data')